## Filtro emoji según estado de ánimo

#### Comprobación de GPU

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.test.gpu_device_name()

Num GPUs Available:  1


I0000 00:00:1731548395.762875   19547 gpu_device.cc:2022] Created device /device:GPU:0 with 683 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5


'/device:GPU:0'

#### Filtro

In [6]:
import cv2
from deepface import DeepFace
import numpy as np
import cairosvg

# Factor de escala para agrandar el ícono
icon_scale_factor = 1.2

# Función para cargar y convertir un archivo SVG en un PNG compatible con OpenCV
def load_svg_to_opencv(path, width, height):
    # Ajustar el ancho y alto al factor de escala
    width = int(width * icon_scale_factor)
    height = int(height * icon_scale_factor)
    
    # Convierte el SVG a PNG en memoria
    png_data = cairosvg.svg2png(url=path, output_width=width, output_height=height)
    
    # Convertir PNG en datos de imagen para OpenCV
    nparr = np.frombuffer(png_data, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_UNCHANGED)  # Lee la imagen con transparencia (canal alpha)
    return img

# Diccionario de iconos de emociones usando SVGs
emotion_icons = {
    "happy": "/home/m4rc/Desktop/VC/Ejercicios Practicas/Practica5/assets/emojis/1f604.svg",
    "sad": "/home/m4rc/Desktop/VC/Ejercicios Practicas/Practica5/assets/emojis/1f622.svg",
    "angry": "/home/m4rc/Desktop/VC/Ejercicios Practicas/Practica5/assets/emojis/1f621.svg",
    "surprise": "/home/m4rc/Desktop/VC/Ejercicios Practicas/Practica5/assets/emojis/1f62f.svg",
    "neutral": "/home/m4rc/Desktop/VC/Ejercicios Practicas/Practica5/assets/emojis/1f610.svg"
}

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Iniciar captura de video
cap = cv2.VideoCapture(0)

while True:
    # Capturar fotograma
    ret, frame = cap.read()
    
    # Duplicar el fotograma original para tener uno sin el efecto
    original_frame = frame.copy()

    # Convertir el fotograma a escala de grises
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Convertir el fotograma en escala de grises a RGB
    rgb_frame = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2RGB)

    # Detectar rostros en el fotograma
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extraer la región de interés del rostro
        face_roi = rgb_frame[y:y + h, x:x + w]

        # Realizar el análisis de emociones en la ROI
        result = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)

        # Obtener la emoción dominante
        emotion = result[0]['dominant_emotion']
        
        # Verificar si hay un ícono para la emoción detectada y cargarlo
        if emotion in emotion_icons:
            icon_path = emotion_icons[emotion]
            
            # Cargar y redimensionar el ícono SVG a PNG con un tamaño más grande
            icon_resized = load_svg_to_opencv(icon_path, w, h)

            # Calcular la posición para centrar el ícono más grande en el rostro
            y_offset = y - int((icon_resized.shape[0] - h) / 2)
            x_offset = x - int((icon_resized.shape[1] - w) / 2)

            # Ajustar los límites si están fuera de los límites del fotograma
            y1, y2 = max(y_offset, 0), min(y_offset + icon_resized.shape[0], frame.shape[0])
            x1, x2 = max(x_offset, 0), min(x_offset + icon_resized.shape[1], frame.shape[1])
            icon_y1, icon_y2 = 0, y2 - y1
            icon_x1, icon_x2 = 0, x2 - x1

            # Superponer el ícono en la imagen principal
            for c in range(0, 3):  # Aplicar sobre cada canal de color
                frame[y1:y2, x1:x2, c] = np.where(
                    icon_resized[icon_y1:icon_y2, icon_x1:icon_x2, 3] > 0,  # Verifica el canal alpha
                    icon_resized[icon_y1:icon_y2, icon_x1:icon_x2, c],
                    frame[y1:y2, x1:x2, c]
                )

    # Mostrar el fotograma con efecto y el original en dos ventanas separadas
    cv2.imshow('Real-time Emotion Detection (with effect)', frame)
    cv2.imshow('Original Camera View', original_frame)

    # Presionar 'q' para salir
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la captura y cerrar todas las ventanas
cap.release()
cv2.destroyAllWindows()
